In [99]:
# Import the same dependencies as in the original model's workbook.
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
from google.colab import files

#  Import and read the original data again.
import pandas as pd
alphasoup = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
alphasoup.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


**We've run into a problem. Alphabet Soup won't want our model unless it reaches a predictive accuracy of 75%, and our previous model fell short of that. We should come up with some ways to improve it.**

# ***OPTIMIZATION ITERATION 1: DROP MORE COLUMNS***

In [61]:
# Drop the non-beneficial ID columns 'EIN' and 'NAME' again.
alphasoup2=alphasoup.drop(['EIN', 'NAME'], axis=1)

In [62]:
#Let's also drop special considerations, organization, use case, and status.
alphasoup3=alphasoup2.drop(['ORGANIZATION','USE_CASE','STATUS','SPECIAL_CONSIDERATIONS'], axis=1)
alphasoup3.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,0,5000,1
1,T3,Independent,C2000,1-9999,108590,1
2,T5,CompanySponsored,C3000,0,5000,0
3,T3,CompanySponsored,C2000,10000-24999,6692,1
4,T3,Independent,C1000,100000-499999,142590,1


In [63]:
#Preprocess this data again. Start with nunique.
alphasoup3.nunique()

APPLICATION_TYPE      17
AFFILIATION            6
CLASSIFICATION        71
INCOME_AMT             9
ASK_AMT             8747
IS_SUCCESSFUL          2
dtype: int64

In [64]:
#Prepare application bins, cut off, and replace.
type_values=alphasoup3["APPLICATION_TYPE"].value_counts()
application_types_to_replace = list(type_values.iloc[8:].index)
application_cutoff=500
for app in application_types_to_replace:
    alphasoup3['APPLICATION_TYPE'] = alphasoup3['APPLICATION_TYPE'].replace(app,"Other")

alphasoup3['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [65]:
#Ditto for classification. Bin, cut off, and replace.
classification_values = alphasoup3['CLASSIFICATION'].value_counts()
classification_values

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [66]:
classifications_to_replace = list(classification_values.iloc[8:].index)
classification_cutoff=200
for cls in classifications_to_replace:
    alphasoup3['CLASSIFICATION'] = alphasoup3['CLASSIFICATION'].replace(cls,"Other")


alphasoup3['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1003
C7000      777
C1700      287
C4000      194
Name: CLASSIFICATION, dtype: int64

In [67]:
#Make another dummies.
dummies = pd.get_dummies(alphasoup3)
dummies

,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,CLASSIFICATION_Other,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,5000,1,0,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,108590,1,0,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,5000,0,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
3,6692,1,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,142590,1,0,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,5000,0,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0
34295,5000,0,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0
34296,5000,0,0,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
34297,5000,1,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0


In [68]:
#Features, target, and splitting.
y=dummies["IS_SUCCESSFUL"]
X=dummies.drop("IS_SUCCESSFUL",axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [69]:
# Create a StandardScaler instance, fit it, and scale our data.
scaler = StandardScaler()

X_scaler = scaler.fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [70]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features = X_train.shape[1]
first_hidden_nodes = 64
second_hidden_nodes = 32

nn = tf.keras.models.Sequential()

# First hidden layer.
nn.add(tf.keras.layers.Dense(units=first_hidden_nodes, input_dim=input_features, activation="relu"))

# Second hidden layer.
nn.add(tf.keras.layers.Dense(units=second_hidden_nodes, activation="relu"))

# Output layer.
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Review our model's structure.
nn.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 64)                2240      
                                                                 
 dense_13 (Dense)            (None, 32)                2080      
                                                                 
 dense_14 (Dense)            (None, 1)                 33        
                                                                 
Total params: 4353 (17.00 KB)
Trainable params: 4353 (17.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [71]:
# Compile the model, train it, and be patient.
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 3s 2ms/step - loss: 0.5758 - accuracy: 0.7220
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5651 - accuracy: 0.7280
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5634 - accuracy: 0.7277
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5615 - accuracy: 0.7285
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5608 - accuracy: 0.7289
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5608 - accuracy: 0.7278
Epoch 7/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5598 - accuracy: 0.7284
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5600 - accuracy: 0.7298
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5594 - accuracy: 0.7300
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5595 - accura

In [72]:
# Evaluate the model using the test data.
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5656 - accuracy: 0.7240 - 478ms/epoch - 2ms/step
Loss: 0.5655567646026611, Accuracy: 0.7239649891853333


Looks like reducing columns actually set us back. Let's try something else.

# ***OPTIMIZATION ITERATION 2: ADD MORE HIDDEN LAYERS***

In [73]:
#Let's head back to alphasoup2 and bring back the original type_values.
type_values=alphasoup2["APPLICATION_TYPE"].value_counts()
type_values

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [74]:
# Choose a cutoff value and create a list of application types to be replaced. Use the variable name `application_types_to_replace`.
application_types_to_replace = list(type_values[type_values < 528].index)
application_cutoff=528
# Make the replacement in our DataFrame. Display the column to confirm successful binning.
for app in application_types_to_replace:
    alphasoup2['APPLICATION_TYPE'] = alphasoup2['APPLICATION_TYPE'].replace(app,"Other")

alphasoup2['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [75]:
# Look at CLASSIFICATION value counts for binning.
classification_values = alphasoup2['CLASSIFICATION'].value_counts()
classification_values

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [76]:
# Choose a cutoff value and create a list of classifications to be replaced. Use the variable name `classifications_to_replace`.
classifications_to_replace = list(classification_values[classification_values<287].index)
classification_cutoff=287
# Make the replacement in our DataFrame. Display the column to confirm successful binning.
for cls in classifications_to_replace:
    alphasoup2['CLASSIFICATION'] = alphasoup2['CLASSIFICATION'].replace(cls,"Other")


alphasoup2['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1197
C7000      777
C1700      287
Name: CLASSIFICATION, dtype: int64

In [77]:
#Time to make another dummies!
dummies = pd.get_dummies(alphasoup2)
dummies

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
34295,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
34296,1,5000,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34297,1,5000,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0


In [78]:
# Split our preprocessed data into our features and target arrays.
y=dummies["IS_SUCCESSFUL"]
X=dummies.drop("IS_SUCCESSFUL",axis=1)
# Split the preprocessed data into a training and testing dataset.
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [79]:
# Create a StandardScaler instance, fit it, and scale our data.
scaler = StandardScaler()

X_scaler = scaler.fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [80]:
# Define the model as before, but let's add three more sets of hidden nodes.
input_features = X_train.shape[1]
first_hidden_nodes = 64
second_hidden_nodes = 32
third_hidden_nodes = 16
fourth_hidden_nodes = 8
fifth_hidden_nodes = 4
nn = tf.keras.models.Sequential()

# First hidden layer.
nn.add(tf.keras.layers.Dense(units=first_hidden_nodes, input_dim=input_features, activation="relu"))

# Second hidden layer.
nn.add(tf.keras.layers.Dense(units=second_hidden_nodes, activation="relu"))

# Third hidden layer.
nn.add(tf.keras.layers.Dense(units=third_hidden_nodes, activation="relu"))

# Fourth hidden layer.
nn.add(tf.keras.layers.Dense(units=fourth_hidden_nodes, activation="relu"))

# Fifth hidden layer.
nn.add(tf.keras.layers.Dense(units=fifth_hidden_nodes, activation="relu"))

# Output layer.
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Review our model's structure.
nn.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_15 (Dense)            (None, 64)                2944      
                                                                 
 dense_16 (Dense)            (None, 32)                2080      
                                                                 
 dense_17 (Dense)            (None, 16)                528       
                                                                 
 dense_18 (Dense)            (None, 8)                 136       
                                                                 
 dense_19 (Dense)            (None, 4)                 36        
                                                                 
 dense_20 (Dense)            (None, 1)                 5         
                                                                 
Total params: 5729 (22.38 KB)
Trainable params: 5729 (

In [81]:
# Compile the model, train it, and be patient.
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 3s 2ms/step - loss: 0.5865 - accuracy: 0.7114
Epoch 2/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5648 - accuracy: 0.7289
Epoch 3/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5605 - accuracy: 0.7313
Epoch 4/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5583 - accuracy: 0.7332
Epoch 5/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5577 - accuracy: 0.7332
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5556 - accuracy: 0.7329
Epoch 7/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5586 - accuracy: 0.7337
Epoch 8/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5562 - accuracy: 0.7332
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5545 - accuracy: 0.7338
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5527 - accura

In [82]:
# Evaluate the model using the test data.
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5558 - accuracy: 0.7266 - 518ms/epoch - 2ms/step
Loss: 0.5557510256767273, Accuracy: 0.7266472578048706


Not quite. How about one more time?

# ***OPTIMIZATION ITERATION 3: RAISE THE MINIMA OF BOTH SETS OF BINS***

In [83]:
#Re-import our data and drop EIN and NAME.
alphasoup4 = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
alphasoup4=alphasoup4.drop(['EIN', 'NAME'], axis=1)

In [84]:
type_values=alphasoup4["APPLICATION_TYPE"].value_counts()
type_values

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [85]:
# Choose a cutoff value and create a list of application types to be replaced. Use the variable name `application_types_to_replace`.
application_types_to_replace = list(type_values[type_values < 1065].index)
application_cutoff=1065
# Make the replacement in our DataFrame. Display the column to confirm successful binning.
for app in application_types_to_replace:
    alphasoup4['APPLICATION_TYPE'] = alphasoup4['APPLICATION_TYPE'].replace(app,"Other")

alphasoup4['APPLICATION_TYPE'].value_counts()

T3       27037
Other     2266
T4        1542
T6        1216
T5        1173
T19       1065
Name: APPLICATION_TYPE, dtype: int64

In [86]:
classification_values = alphasoup4['CLASSIFICATION'].value_counts()
classification_values

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [87]:
# Choose a cutoff value and create a list of classifications to be replaced. Use the variable name `classifications_to_replace`.
classifications_to_replace = list(classification_values[classification_values<1883].index)
classification_cutoff=1883
# Make the replacement in our DataFrame. Display the column to confirm successful binning.
for cls in classifications_to_replace:
    alphasoup4['CLASSIFICATION'] = alphasoup4['CLASSIFICATION'].replace(cls,"Other")


alphasoup4['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [88]:
#Dummies! Again!
dummies = pd.get_dummies(alphasoup4)
dummies

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,AFFILIATION_CompanySponsored,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,1,0,0,0,1,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,5000,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34295,1,5000,0,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,1,0
34296,1,5000,0,0,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
34297,1,5000,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0


In [89]:
# Split our preprocessed data into our features and target arrays.
y=dummies["IS_SUCCESSFUL"]
X=dummies.drop("IS_SUCCESSFUL",axis=1)
# Split the preprocessed data into a training and testing dataset.
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [90]:
# Create a StandardScaler instance, fit it, and scale our data.
scaler = StandardScaler()

X_scaler = scaler.fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [91]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features = X_train.shape[1]
first_hidden_nodes = 64
second_hidden_nodes = 32

nn = tf.keras.models.Sequential()

# First hidden layer.
nn.add(tf.keras.layers.Dense(units=first_hidden_nodes, input_dim=input_features, activation="relu"))

# Second hidden layer.
nn.add(tf.keras.layers.Dense(units=second_hidden_nodes, activation="relu"))

# Output layer.
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Review our model's structure.
nn.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_21 (Dense)            (None, 64)                2624      
                                                                 
 dense_22 (Dense)            (None, 32)                2080      
                                                                 
 dense_23 (Dense)            (None, 1)                 33        
                                                                 
Total params: 4737 (18.50 KB)
Trainable params: 4737 (18.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [92]:
# Compile the model, train it, and be patient.
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 4s 3ms/step - loss: 0.5733 - accuracy: 0.7223
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5591 - accuracy: 0.7289
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5561 - accuracy: 0.7307
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5549 - accuracy: 0.7315
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5533 - accuracy: 0.7327
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5528 - accuracy: 0.7325
Epoch 7/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5514 - accuracy: 0.7343
Epoch 8/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5512 - accuracy: 0.7341
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5507 - accuracy: 0.7341
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5500 - accura

In [93]:
# Evaluate the model using the test data.
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5609 - accuracy: 0.7251 - 496ms/epoch - 2ms/step
Loss: 0.5609342455863953, Accuracy: 0.7251312136650085


Unfortunately, we still have even more work to do before Alphabet Soup will accept our model.

In [101]:
#For future reference, let's save our optimizations to another h5.
nn.save('AlphabetSoupCharity_Optimization.h5')
files.download('AlphabetSoupCharity_Optimization.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>